In [12]:
# Split Dataset

import os
import shutil

original_dataset_dir = '/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec88'
classes_list = os.listdir(original_dataset_dir)

base_dir = '/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# train,validation,test 폴더 하위에 각각 클래스 목록 폴더 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir,clss))
    os.mkdir(os.path.join(test_dir,clss))
    

# [데이터 분할과 클래스별 데이터 수 확인]
import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir,clss)
    fnames = os.listdir(path)
    
    train_size = math.floor(len(fnames)*0.8)
    test_size = math.floor(len(fnames)*0.2)
    
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src,dst)
        
    test_fnames = fnames[train_size:(train_size+test_size)]
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss),fname)
        shutil.copyfile(src,dst)

Train size( transistor_damaged_case ):  8
Test size( transistor_damaged_case ):  2
Train size( leather_good ):  221
Test size( leather_good ):  55
Train size( tile_gray_stroke ):  12
Test size( tile_gray_stroke ):  3
Train size( capsule_crack ):  18
Test size( capsule_crack ):  4
Train size( cable_missing_wire ):  8
Test size( cable_missing_wire ):  2
Train size( grid_bent ):  9
Test size( grid_bent ):  2
Train size( hazelnut_good ):  344
Test size( hazelnut_good ):  86
Train size( carpet_thread ):  15
Test size( carpet_thread ):  3
Train size( transistor_cut_lead ):  8
Test size( transistor_cut_lead ):  2
Train size( wood_hole ):  8
Test size( wood_hole ):  2
Train size( tile_crack ):  13
Test size( tile_crack ):  3
Train size( pill_contamination ):  16
Test size( pill_contamination ):  4
Train size( hazelnut_crack ):  14
Test size( hazelnut_crack ):  3
Train size( wood_good ):  212
Test size( wood_good ):  53
Train size( bottle_contamination ):  16
Test size( bottle_contamination ): 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, SubsetRandomSampler, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for dataset
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted/train', transform=transform)
test_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/splitted/test', transform=transform)

# Define initial labeled dataset
num_initial_samples = 1000
labeled_indices = torch.randperm(len(train_dataset))[:num_initial_samples]
labeled_dataset = torch.utils.data.Subset(train_dataset, labeled_indices)

# Define unlabeled dataset
unlabeled_indices = torch.arange(len(train_dataset))[~torch.eq(torch.arange(len(train_dataset)).unsqueeze(1), labeled_indices).any(1)]
unlabeled_dataset = torch.utils.data.Subset(train_dataset, unlabeled_indices)

# Define data loaders
labeled_loader = DataLoader(labeled_dataset, batch_size=32, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torch.nn as nn
from torchvision.models import resnet18

# Define ResNet model
class ResNet18(nn.Module):
    def __init__(self, num_classes=88):
        super(ResNet18, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet18.conv1(x)
        x = self.resnet18.bn1(x)
        x = self.resnet18.relu(x)
        x = self.resnet18.maxpool(x)

        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)

        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = ResNet18(num_classes=88).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train(model, optimizer, labeled_loader):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    total = 0
    for images, labels in labeled_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_acc += (predicted == labels).sum().item()
    train_loss /= len(labeled_loader)
    train_acc /= total
    return train_loss, train_acc

/home/iai/anaconda3/envs/jwanaconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Active learning loop
num_queries = 10
batch_size = 32

for query in range(num_queries):
    print("Query number:", query+1)
    
    # Train model on labeled dataset
    train_loss, train_acc = train(model, optimizer, labeled_loader)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    # Evaluate model on test dataset
    with torch.no_grad():
        test_loss = 0.0
        test_acc = 0.0
        total = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            test_acc += (predicted == labels).sum().item()
        test_loss /= len(test_loader)
        test_acc /= total
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    
    # Make model predictions on unlabeled dataset
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in unlabeled_loader:
            images = images.to(device)  # Move data to GPU
            output = model(images)
            predictions.append(output)
    predictions = torch.cat(predictions, dim=0)

    # Select samples to label using entropy-based method
    entropy = -torch.sum(torch.softmax(predictions, dim=1) * torch.log(torch.softmax(predictions, dim=1)), dim=1)
    _, idx = torch.topk(entropy, k=batch_size)
    labeled_indices = torch.cat([labeled_indices, unlabeled_indices[idx]])
    unlabeled_indices = unlabeled_indices[~torch.eq(unlabeled_indices.unsqueeze(1), unlabeled_indices[idx]).any(1)]
    
    # Update labeled and unlabeled datasets
    labeled_dataset = torch.utils.data.Subset(train_dataset, labeled_indices)
    unlabeled_dataset = torch.utils.data.Subset(train_dataset, unlabeled_indices)

    # Update labeled and unlabeled data loaders
    labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)
    unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)

Query number: 1
Train Loss: 1.9573, Train Accuracy: 0.6830
Test Loss: 4.2235, Test Accuracy: 0.1620
Query number: 2
Train Loss: 1.2845, Train Accuracy: 0.7432
Test Loss: 4.6284, Test Accuracy: 0.1302
Query number: 3
Train Loss: 1.1977, Train Accuracy: 0.7491
Test Loss: 4.6069, Test Accuracy: 0.1504
Query number: 4
Train Loss: 1.1094, Train Accuracy: 0.7600
Test Loss: 4.6941, Test Accuracy: 0.1755
Query number: 5
Train Loss: 1.1173, Train Accuracy: 0.7535
Test Loss: 4.8147, Test Accuracy: 0.1553
Query number: 6
Train Loss: 1.0713, Train Accuracy: 0.7552
Test Loss: 4.8853, Test Accuracy: 0.1553
Query number: 7
Train Loss: 0.9972, Train Accuracy: 0.7617
Test Loss: 5.0944, Test Accuracy: 0.1446
Query number: 8
Train Loss: 0.9525, Train Accuracy: 0.7786
Test Loss: 4.7690, Test Accuracy: 0.1688
Query number: 9
Train Loss: 1.0485, Train Accuracy: 0.7516
Test Loss: 4.9597, Test Accuracy: 0.1553
Query number: 10
Train Loss: 1.0366, Train Accuracy: 0.7547
Test Loss: 5.1657, Test Accuracy: 0.1649